# Class IV - An introduction to GasModels.jl

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of the [GasModels.jl](https://github.com/lanl-ansi/GasModels.jl) package. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) and [Class II - An introduction  to JuMP](Class%20II%20-%20An%20introduction%20to%20JuMP.ipynb) first.

As in Class II, run the following magic sauce to check we're good to go.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Excellent! Everything is good to go!


### Background 

This [presentation](https://github.com/lanl-ansi/tutorial-grid-science-2019/blob/master/assets/infrastructure_optimization_in_julia.pdf) provides motivation and context for this notebook.

Some additional informaiton is also available at,
* [GasModels Documentation](https://lanl-ansi.github.io/GasModels.jl/stable/)


### Working with the Network Model

A 40-node gas network models is provided with this tutorial.  The MATLAB-like input data files can be viewed [here](../edit/data/gaslib40.m).

Similar to PowerModels, the `parse_file` function from GasModels is used to load a text files into the GasModels data model, 

In [2]:
using GasModels

data = GasModels.parse_file("data/gaslib40.m")

┌ Info: Recompiling stale cache file /Users/carleton/.julia/compiled/v1.1/GasModels/5VKJw.ji for GasModels [5e113713-6c35-5477-b766-e1109486666f]
└ @ Base loading.jl:1184


[warn | InfrastructureModels]: Matlab parser skipping the following line:
  end
[warn | GasModels]: no case version found in .m file.  The file seems to be missing "mgc.version = ..."


Dict{String,Any} with 21 entries:
  "gas_specific_gravity"         => 0.6
  "temperature"                  => 273.15
  "source_type"                  => ".m"
  "name"                         => "gaslib40"
  "source_version"               => "0.0.0+"
  "multinetwork"                 => false
  "gas_molar_mass"               => 0.0185674
  "compressor"                   => Dict{String,Any}("41"=>Dict{String,Any}("t_…
  "consumer"                     => Dict{String,Any}("24"=>Dict{String,Any}("ql…
  "standard_density"             => 1.0
  "baseQ"                        => 604.167
  "compressibility_factor"       => 0.8
  "specific_heat_capacity_ratio" => 1.4
  "producer"                     => Dict{String,Any}("1"=>Dict{String,Any}("qgm…
  "per_unit"                     => true
  "junction"                     => Dict{String,Any}("32"=>Dict{String,Any}("ju…
  "pipe"                         => Dict{String,Any}("32"=>Dict{String,Any}("le…
  "baseP"                        => 8101325
  "sound

In this case the file parser generated warning messages.  

The second warning block message indicates that the data file did not have any version information. Still the parser will load this data.

Now, lets look at some of the network data.  The data for the first pipe and compressor can be viewed as follows,

In [3]:
data["connection"]["1"]

ArgumentError: ArgumentError: invalid index: 1 of type String

In [4]:
data["connection"]["100000"]

ArgumentError: ArgumentError: invalid index: 100000 of type String

Here, connection can either be of type "pipe" in which case it has a length, diameter, and friction_factor associated with it or can be a compressor with a different set of fields as shown above.

The first junction in the network can be viewed as follows,

In [5]:
data["junction"]["1"]

Dict{String,Any} with 7 entries:
  "junction_type" => 0
  "status"        => 1
  "junction_i"    => 1
  "pmax"          => 1.0
  "p_nominal"     => 3.10133e6
  "pmin"          => 0.382817
  "index"         => 1

That's great, but looking at components one-by-one can get boring fast.

All InfrastructureModels packages provide a `print_summary` function that prints a table-like summary of the network data to the terminal.

In [6]:
GasModels.print_summary(data)

Metadata
  R: 8.314
  baseP: 8101325
  baseQ: 604.167
  compressibility_factor: 0.800
  connection: [(0)]
  gas_molar_mass: 0.019
  gas_specific_gravity: 0.600
  multinetwork: false
  name: gaslib40
  per_unit: true
  sound_speed: 312.805
  source_type: .m
  source_version: 0.0.0+
  specific_heat_capacity_ratio: 1.400
  standard_density: 1.000
  temperature: 273.150

Table Counts
  junction: 46
  producer: 3
  consumer: 29
  compressor: 12
  pipe: 39


Table: junction
          junction_i,  pmin,  pmax,  p_nominal
       0:          0, 0.013, 1.000, 101325.000
       1:          1, 0.383, 1.000, 3.101325e6
       2:          2, 0.383, 1.000, 3.101325e6
       3:          3, 0.013, 1.000, 101325.000
       4:          4, 0.013, 1.000, 101325.000
       5:          5, 0.383, 1.000, 3.101325e6
       6:          6, 0.013, 1.000, 101325.000
       7:          7, 0.013, 1.000, 101325.000
       8:          8, 0.013, 1.000, 101325.000
       9:          9, 0.013, 1.000, 101325.000
      10: 

### Solving the Gas Flow Problem (GF)

Before we can solve a gas flow problem, we need a solver.  The gas flow problem as defined in GasModels.jl is a Mixed-Integer Nonlinear Program (MINLP). Hence, we require an MINLP solver; in this case we will use [Pavito](https://github.com/JuliaOpt/Pavito.jl). This MINLP solver requires two other solvers (1) a mip-solver and (2) a continuous NLP solver. We shall use GLPK and Ipopt as the mip solver and NLP solvers, respectively. 

In [7]:
using GLPKMathProgInterface
using Ipopt
using Pavito

glpk_solver = GLPKSolverMIP()
ipopt_solver = IpoptSolver(tol=1e-6, print_level=0)

pavito_solver = PavitoSolver(mip_solver=glpk_solver, cont_solver=ipopt_solver, mip_solver_drives=false, log_level=1)

result = run_gf(data, MINLPGasModel, pavito_solver)

┌ Info: Recompiling stale cache file /Users/carleton/.julia/compiled/v1.1/Pavito/TnlEj.ji for Pavito [cd433a01-47d1-575d-afb7-6db927ee8d8f]
└ @ Base loading.jl:1184



MINLP has a linear objective, 199 variables (102 integer), 565 constraints (156 nonlinear)

Pavito started, using iterative method...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************


Iter. | Best feasible  | Best bound     | Rel. gap    | Time (s)   
    1 |           +Inf |  +0.000000e+00 |       >1000 |   1.871e+00

Pavito finished...

Status                 Optimal
Objective value        0.00000
Objective bound        0.00000
Objective gap          0.00000
Iterations                   1
Total time             2.00626 sec
MIP total time         0.62883 sec
NLP total time         0.26152 sec



Dict{String,Any} with 8 entries:
  "solve_time"   => 2.00626
  "status"       => :Optimal
  "objective"    => 0.0
  "solver"       => "PavitoSolver"
  "solution"     => Dict{String,Any}("baseQ"=>604.167,"junction"=>Dict{String,A…
  "data"         => Dict{String,Any}("name"=>"gaslib40","compressor_count"=>12,…
  "machine"      => Dict("cpu"=>"Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz","me…
  "objective_lb" => 0.0

The result object contains a variety of useful information about the optimization problem solved, including the objective value and wall clock runtime.  Detailed documentation of the result dictionary is available [here](https://lanl-ansi.github.io/GasModels.jl/stable/result-data/), however most interesting point is the `solution` data.

In [8]:
result["solution"]

Dict{String,Any} with 7 entries:
  "baseQ"        => 604.167
  "junction"     => Dict{String,Any}("32"=>Dict{String,Any}("p"=>0.799014),"29"…
  "pipe"         => Dict{String,Any}("32"=>Dict{String,Any}("f"=>0.100659),"29"…
  "baseP"        => 8101325
  "multinetwork" => false
  "per_unit"     => true
  "compressor"   => Dict{String,Any}("41"=>Dict{String,Any}("f"=>0.163709,"rati…

Not every useful, lets try `print_summary`,

In [9]:
GasModels.print_summary(result["solution"])

Metadata
  baseP: 8101325
  baseQ: 604.167
  multinetwork: false
  per_unit: true

Table Counts
  junction: 46
  compressor: 12
  pipe: 39


Table: junction
              p
       0: 0.700
       1: 0.697
       2: 0.610
       3: 0.585
       4: 0.817
       5: 0.693
       6: 0.674
       7: 0.648
       8: 0.589
       9: 0.589
      10: 0.670
      11: 0.629
      12: 0.710
      13: 0.709
      14: 0.178
      15: 0.688
      16: 0.688
      17: 0.816
      18: 0.798
      19: 0.668
      20: 0.603
      21: 0.720
      22: 0.680
      23: 0.206
      24: 0.586
      25: 0.693
      26: 0.209
      27: 0.813
      28: 0.686
      29: 0.728
      30: 0.823
      31: 0.824
      32: 0.799
      33: 0.734
      34: 0.719
      35: 0.734
      36: 0.733
      37: 0.687
      38: 0.867
      39: 0.853
  100021: 0.734
  200002: 0.734
  300001: 0.867
  400037: 0.813
  500005: 0.853
  600013: 0.799


Table: compressor
               f, ratio
      39:  0.092, 1.184
      40:  0.034, 1.126